# Deploy Mnist model with Triton

## Prerequisites

 * A kubernetes cluster with kubectl configured
%%writefile rclone.conf
[s3]
type = s3
provider = minio
env_auth = false
access_key_id = minioadmin
secret_access_key = minioadmin
endpoint = http://localhost:8090 * rclone
 * curl

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

We will assume that ambassador (or Istio) ingress is port-forwarded to `localhost:8003`

## Setup MinIO

Use the provided [notebook](https://docs.seldon.io/projects/seldon-core/en/latest/examples/minio_setup.html) to install Minio in your cluster.
Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/minio_setup.html).

We will assume that MinIO service is port-forwarded to `localhost:8090`

In [84]:
%%writefile rclone.conf
[s3]
type = s3
provider = minio
env_auth = false
access_key_id = admin@seldon.io
secret_access_key = 12341234
endpoint = http://localhost:8090

Overwriting rclone.conf


In [85]:
%%writefile secret.yaml
apiVersion: v1
kind: Secret
metadata:
  name: seldon-rclone-secret
type: Opaque
stringData:
  RCLONE_CONFIG_S3_TYPE: s3
  RCLONE_CONFIG_S3_PROVIDER: minio
  RCLONE_CONFIG_S3_ENV_AUTH: "false"
  RCLONE_CONFIG_S3_ACCESS_KEY_ID: "admin@seldon.io"
  RCLONE_CONFIG_S3_SECRET_ACCESS_KEY: "12341234"
  RCLONE_CONFIG_S3_ENDPOINT: http://minio.minio-system.svc.cluster.local:9000

Overwriting secret.yaml


In [86]:
!kubectl apply -f secret.yaml

secret/seldon-rclone-secret configured


# Add Triton cofniguration

In [1]:
%%writefile triton-mlflow/mnist-save-model/mnist/config.pbtxt 
name: "mnist"
platform: "tensorflow_savedmodel"
max_batch_size: 100
dynamic_batching { preferred_batch_size: [ 50 ]}
instance_group [ { count: 2 }]
input [
  {
    name: "conv2d_input"
    data_type: TYPE_FP32
    dims: [ 28, 28, 1 ]
  }
]
output [
  {
    name: "dense_1"
    data_type: TYPE_FP32
    dims: [ 10 ]
  }
]

Overwriting triton-mlflow/mnist-save-model/mnist/config.pbtxt


In [2]:
%%writefile triton-mlflow/mnist-log-model/mnist/config.pbtxt 
name: "mnist"
platform: "tensorflow_savedmodel"
max_batch_size: 100
dynamic_batching { preferred_batch_size: [ 50 ]}
instance_group [ { count: 2 }]
input [
  {
    name: "conv2d_input"
    data_type: TYPE_FP32
    dims: [ 28, 28, 1 ]
  }
]
output [
  {
    name: "dense_1"
    data_type: TYPE_FP32
    dims: [ 10 ]
  }
]

Writing triton-mlflow/mnist-log-model/mnist/config.pbtxt


# Copy model into S3

In [89]:
!rclone -vvv --config="rclone.conf" copy triton-mlflow s3://triton-mlflow

2021/10/13 11:55:31 DEBUG : rclone: Version "v1.56.2" starting with parameters ["/home/rskolasinski/.asdf/installs/rclone/1.56.2/bin/rclone" "-vvv" "--config=rclone.conf" "copy" "triton-mlflow" "s3://triton-mlflow"]
2021/10/13 11:55:31 DEBUG : Creating backend with remote "triton-mlflow"
2021/10/13 11:55:31 DEBUG : Using config file from "/home/rskolasinski/work/seldon-core/examples/mnist_triton_e2e/rclone.conf"
2021/10/13 11:55:31 DEBUG : fs cache: renaming cache item "triton-mlflow" to be canonical "/home/rskolasinski/work/seldon-core/examples/mnist_triton_e2e/triton-mlflow"
2021/10/13 11:55:31 DEBUG : Creating backend with remote "s3://triton-mlflow"
2021/10/13 11:55:31 DEBUG : fs cache: renaming cache item "s3://triton-mlflow" to be canonical "s3:triton-mlflow"
2021/10/13 11:55:31 DEBUG : mnist-save-model/mnist/config.pbtxt: Modification times differ by -6m34.559941588s: 2021-10-13 11:55:31.216961083 +0100 BST, 2021-10-13 11:48:56.657019495 +0100 BST
2021/10/13 11:55:31 DEBUG : mni

In [90]:
!rclone --config="rclone.conf" ls s3:

    15048 triton-mlflow/mnist-log-model/mnist/1/model.savedmodel/keras_metadata.pb
    15048 triton-mlflow/mnist-log-model/mnist/1/model.savedmodel/model/keras_metadata.pb
   157755 triton-mlflow/mnist-log-model/mnist/1/model.savedmodel/model/saved_model.pb
  7216711 triton-mlflow/mnist-log-model/mnist/1/model.savedmodel/model/variables/variables.data-00000-of-00001
     2303 triton-mlflow/mnist-log-model/mnist/1/model.savedmodel/model/variables/variables.index
   157755 triton-mlflow/mnist-log-model/mnist/1/model.savedmodel/saved_model.pb
  7216711 triton-mlflow/mnist-log-model/mnist/1/model.savedmodel/variables/variables.data-00000-of-00001
     2303 triton-mlflow/mnist-log-model/mnist/1/model.savedmodel/variables/variables.index
      321 triton-mlflow/mnist-log-model/mnist/config.pbtxt
    15048 triton-mlflow/mnist-save-model/mnist/1/model.savedmodel/keras_metadata.pb
    15048 triton-mlflow/mnist-save-model/mnist/1/model.savedmodel/model/keras_metadata.pb
   157755 triton-mlflow/m

# Deploy and Test model

## Deploy

In [91]:
%%writefile deployment.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: mnist-save-model
  namespace: seldon
spec:
  name: default
  predictors:
  - graph:
      implementation: TRITON_SERVER
      logger:
        mode: all
      modelUri: s3:triton-mlflow/mnist-save-model
      envSecretRefName: seldon-rclone-secret
      name: mnist
      type: MODEL
    name: default
    replicas: 1
  protocol: kfserving
    
---

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: mnist-log-model
  namespace: seldon
spec:
  name: default
  predictors:
  - graph:
      implementation: TRITON_SERVER
      logger:
        mode: all
      modelUri: s3:triton-mlflow/mnist-log-model
      envSecretRefName: seldon-rclone-secret
      name: mnist
      type: MODEL
    name: default
    replicas: 1
  protocol: kfserving

Overwriting deployment.yaml


In [92]:
!kubectl apply -f deployment.yaml

seldondeployment.machinelearning.seldon.io/mnist-save-model unchanged
seldondeployment.machinelearning.seldon.io/mnist-log-model unchanged


In [93]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=mnist -o jsonpath='{.items[0].metadata.name}')

deployment "mnist-default-0-mnist" successfully rolled out


## Test

In [98]:
!curl -s http://localhost:8003/seldon/seldon/mnist/v2/models/mnist | jq .

{
  "name": "mnist",
  "versions": [
    "1"
  ],
  "platform": "tensorflow_savedmodel",
  "inputs": [
    {
      "name": "conv2d_input",
      "datatype": "FP32",
      "shape": [
        -1,
        28,
        28,
        1
      ]
    }
  ],
  "outputs": [
    {
      "name": "dense_1",
      "datatype": "FP32",
      "shape": [
        -1,
        10
      ]
    }
  ]
}


In [99]:
!curl -s http://localhost:8003/seldon/seldon/mnist-log-model/v2/models/mnist | jq .

{
  "name": "mnist",
  "versions": [
    "1"
  ],
  "platform": "tensorflow_savedmodel",
  "inputs": [
    {
      "name": "conv2d_input",
      "datatype": "FP32",
      "shape": [
        -1,
        28,
        28,
        1
      ]
    }
  ],
  "outputs": [
    {
      "name": "dense_1",
      "datatype": "FP32",
      "shape": [
        -1,
        10
      ]
    }
  ]
}


In [100]:
!curl -s http://localhost:8003/seldon/seldon/mnist-save-model/v2/models/mnist | jq .

{
  "name": "mnist",
  "versions": [
    "1"
  ],
  "platform": "tensorflow_savedmodel",
  "inputs": [
    {
      "name": "conv2d_input",
      "datatype": "FP32",
      "shape": [
        -1,
        28,
        28,
        1
      ]
    }
  ],
  "outputs": [
    {
      "name": "dense_1",
      "datatype": "FP32",
      "shape": [
        -1,
        10
      ]
    }
  ]
}


In [101]:
import numpy as np
import requests

In [102]:
URL = "http://localhost:8003/seldon/seldon/mnist"
# URL = "http://localhost:8003/seldon/seldon/mnist-save-model"

data = {
    "inputs": [
        {
            "name": "conv2d_input",
            "data": np.random.rand(10, 28, 28, 1).tolist(),
            "datatype": "FP32",
            "shape": [10, 28, 28, 1],
        }
    ]
}

r = requests.post(f"{URL}/v2/models/mnist/infer", json=data)
predictions = np.array(r.json()["outputs"][0]["data"]).reshape(
    r.json()["outputs"][0]["shape"]
)
output = [np.argmax(x) for x in predictions]
output

[8, 8, 8, 8, 2, 8, 8, 8, 8, 2]

In [103]:
URL = "http://localhost:8003/seldon/seldon/mnist-save-model"

data = {
    "inputs": [
        {
            "name": "conv2d_input",
            "data": np.random.rand(10, 28, 28, 1).tolist(),
            "datatype": "FP32",
            "shape": [10, 28, 28, 1],
        }
    ]
}

r = requests.post(f"{URL}/v2/models/mnist/infer", json=data)
predictions = np.array(r.json()["outputs"][0]["data"]).reshape(
    r.json()["outputs"][0]["shape"]
)
output = [np.argmax(x) for x in predictions]
output

[8, 8, 8, 8, 8, 8, 8, 8, 8, 8]

In [104]:
URL = "http://localhost:8003/seldon/seldon/mnist-log-model"

data = {
    "inputs": [
        {
            "name": "conv2d_input",
            "data": np.random.rand(10, 28, 28, 1).tolist(),
            "datatype": "FP32",
            "shape": [10, 28, 28, 1],
        }
    ]
}

r = requests.post(f"{URL}/v2/models/mnist/infer", json=data)
predictions = np.array(r.json()["outputs"][0]["data"]).reshape(
    r.json()["outputs"][0]["shape"]
)
output = [np.argmax(x) for x in predictions]
output

[8, 2, 8, 8, 8, 8, 8, 8, 8, 8]